In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re 
import time

In [2]:
s = Service(r'C:\Users\natas\AppData\Local\Chrome\chromedriver.exe')
#AppData\Local\Chrome\chromedriver.exe')
driver = webdriver.Chrome(r'C:\Users\natas\AppData\Local\Chrome\chromedriver.exe')

In [3]:
def get_by_id(id_punk):
    driver.get("https://www.larvalabs.com/cryptopunks/details/"+str(id_punk))
    time.sleep(np.random.random()*8) 
    content=driver.page_source.encode('utf-8').strip()
    soup= BeautifulSoup(content,"html.parser")
    return soup

In [4]:
#type
def get_type(soup):
    type_punk=soup.findAll("div",{"class": "col-md-10"})
    return type_punk[0].find("a").text

In [5]:
#attributts
def get_attributts(soup):
    list_attr=[]
    attr=soup.findAll("div",{"class": "col-md-4"})
    for i in range(1,len(attr)):
        attr_text=attr[i].find("a").text
        list_attr.append(attr_text)                  
    return list_attr

In [6]:
#regular expressions
def get_value(pattern, line):
    value=""
    match = re.search(pattern,line) 
    if match:
        value=match[0]
    return value    

In [7]:
#users for transaction
def get_user(str_user):
    pattern = "[=]\w+"
    if str_user!=None:
        user = get_value(pattern,str_user['href'])[1:]        
    else:
        user=""
    return user

In [8]:
def get_transaction(transaction_rows):
    list_tran=[]
    all_tran=[]
    df_tran=pd.DataFrame()
    pattern_dol ="[$][^A-Z)]+"
    pattern_ethereums ="[\d\,\.]*"
    pattern_letter ="[A-Z]"
    for element in range(1,len(transaction_rows)):
        list_transaction=[]
        tr=transaction_rows[element].find_all(['th', 'td'])
        _type_tr=tr[0].text
        _from=get_user(tr[1].find("a",href=True))
        _to=get_user(tr[2].find("a",href=True))
        amount_dol=get_value(pattern_dol,tr[3].text)[1:]
        amount_ethereums=get_value(pattern_ethereums,tr[3].text)
        letter=get_value(pattern_letter,tr[3].text)
        _date=tr[4].text
        list_tran.extend([_type_tr,_from,_to,amount_dol,amount_ethereums,letter,_date])
        all_tran.append(list_tran)
        df_tran=df_tran.append({"id_punk":_id,"type_transaction":_type_tr,"from":_from,"to":_to,"amount_dol":amount_dol,"amount_ethereums":amount_ethereums,"letter":letter,"date":_date},ignore_index=True)
    return all_tran, df_tran


In [14]:
dict_transaction={}
dict_punk={}
df_transaction=pd.DataFrame(columns=("id_punk","type_transaction","from","to","amount_dol","amount_ethereums","letter","date"))
df_punk=pd.DataFrame()
for k in range(5393,5395):
    _id=k
    soup_punk=get_by_id(_id)
    types=get_type(soup_punk)
    attr=get_attributts(soup_punk)
    dict_punk[_id]={"type":types,"attr":attr}
    for rr in attr:
        df_punk=df_punk.append({"_id":_id,"_type":types,"attr":rr},ignore_index=True)
#print(dict_punk)

#transaction
    transaction = soup_punk.find_all('tr')
    all_transaction, df_tr=get_transaction(transaction)
    dict_transaction[_id]=all_transaction
    df_transaction=pd.concat([df_transaction,df_tr],ignore_index=True)


In [10]:
len(df_transaction["from"].unique())

5

In [11]:
df_transaction

,id_punk,type_transaction,from,to,amount_dol,amount_ethereums,letter,date
0,5394.0,Bid Withdrawn,0xa7bd745215fa1cee3e93ef2de195c5a53b6c75bb,,"29,036",14,,"Apr 06, 2021"
1,5394.0,Sold,0xa3cb30128532f43fe1c8a9f4c19422aa631307a9,0xd7e077576f13e54dce22da96338d56b7d9ab7c21,"34,666",16.50,,"Apr 06, 2021"
2,5394.0,Bid,0xa7bd745215fa1cee3e93ef2de195c5a53b6c75bb,,"28,875",14,,"Apr 05, 2021"
3,5394.0,Offered,,,<0.01,16.50,,"Apr 02, 2021"
4,5394.0,Bid Withdrawn,0x052564eb0fd8b340803df55def89c25c432f43f4,,"27,018",14,,"Apr 01, 2021"
5,5394.0,Offered,,,"34,639",18,,"Apr 01, 2021"
6,5394.0,Bid,0x052564eb0fd8b340803df55def89c25c432f43f4,,"25,944",14,,"Mar 30, 2021"
7,5394.0,Offered,,,"35,210",19,,"Mar 30, 2021"
8,5394.0,Offered,,,"36,712",19.99,,"Mar 30, 2021"
9,5394.0,Offered,,,"34,910",20.50,,"Mar 29, 2021"


In [12]:
df_punk

,_id,_type,attr
0,5394.0,Male,Mohawk
1,5394.0,Male,Eye Patch


In [13]:
df_punk.to_csv(r"C:\Users\natas\Downloads\punk--.csv")
df_transaction.to_csv(r"C:\Users\natas\Downloads\transaction--.csv")